In [1]:
! echo whoami

whoami


In [3]:
import os
os.getlogin()

'hmnshpl'

In [4]:
scratch_location = rf'/scratch/{os.getlogin()}'
scratch_location

'/scratch/hmnshpl'

In [1]:
# scratch_location = r'/scratch/hmnshpl'
import getpass
scratch_location = rf'/scratch/{getpass.getuser()}'

In [3]:
import pandas as pd
import numpy as np

dataset_name = 'wikipedia'

# Load data and train val test split
graph_df = pd.read_csv('{}/processed_data/{}/ml_{}.csv'.format(scratch_location, dataset_name, dataset_name))
edge_raw_features = np.load('{}/processed_data/{}/ml_{}.npy'.format(scratch_location, dataset_name, dataset_name))
node_raw_features = np.load('{}/processed_data/{}/ml_{}_node.npy'.format(scratch_location, dataset_name, dataset_name))

### Exploring nodes features

In [111]:
el_graph_df = pd.read_csv('{}/sparsified_data/{}/ml_{}.csv'.format(scratch_location, dataset_name, dataset_name))
el_graph_df.head()

,Unnamed: 0.1,u,i,Unnamed: 0,ts,label,idx
0,0,1,8228,0,0.0,0.0,1
1,1,2,8229,1,36.0,0.0,2
2,2,3,8230,3,131.0,0.0,4
3,3,4,8231,6,169.0,0.0,7
4,4,5,8232,8,218.0,0.0,9


In [112]:
graph_df.head()

,Unnamed: 0,u,i,ts,label,idx
0,0,1,8228,0.0,0.0,1
1,1,2,8229,36.0,0.0,2
2,2,2,8229,77.0,0.0,3
3,3,3,8230,131.0,0.0,4
4,4,2,8229,150.0,0.0,5


In [6]:
node_raw_features.shape, type(node_raw_features)

((9228, 172), numpy.ndarray)

In [12]:
lst = []
for i in range(node_raw_features.shape[0]):
    lst.append(max(node_raw_features[i]))

# lst # all raw node features are started with zero 

In [21]:
edge_raw_features.shape, len(edge_raw_features), type(edge_raw_features)

((157475, 172), 157475, numpy.ndarray)

In [22]:
graph_df.shape, type(graph_df)

((157474, 6), pandas.core.frame.DataFrame)

In [23]:
graph_df.head()

,Unnamed: 0,u,i,ts,label,idx
0,0,1,8228,0.0,0.0,1
1,1,2,8229,36.0,0.0,2
2,2,2,8229,77.0,0.0,3
3,3,3,8230,131.0,0.0,4
4,4,2,8229,150.0,0.0,5


In [26]:
graph_df['label'].unique(), graph_df['label'].value_counts(),  

(array([0., 1.]),
 label
 0.0    157257
 1.0       217
 Name: count, dtype: int64)

In [28]:
len(graph_df['u'].unique())

8227

In [33]:
graph_df['i'].value_counts()


i
8412    1937
8735    1914
9167    1184
8341    1175
8333     967
        ... 
8323      84
8524      84
8241      84
8831      84
9166      84
Name: count, Length: 1000, dtype: int64

In [41]:
banned_df = graph_df[graph_df['label'] == 1]

In [42]:
banned_df.head()

,Unnamed: 0,u,i,ts,label,idx
87,87,33,8260,2053.0,1.0,88
701,701,138,8342,17601.0,1.0,702
2571,2571,516,8424,68384.0,1.0,2572
3773,3773,689,8592,95206.0,1.0,3774
3801,3801,690,8592,96027.0,1.0,3802


In [44]:
# banned_df['u'].value_counts()
banned_df['i'].value_counts()

i
9169    6
8261    6
8340    5
8623    4
8369    4
       ..
8815    1
8247    1
8312    1
8929    1
8556    1
Name: count, Length: 139, dtype: int64

In [45]:
banned_df[banned_df['i'] == 9169]

,Unnamed: 0,u,i,ts,label,idx
120662,120662,6850,9169,2011011.0,1.0,120663
140769,140769,7619,9169,2327461.0,1.0,140770
143932,143932,7740,9169,2391790.0,1.0,143933
145566,145566,7799,9169,2423320.0,1.0,145567
145739,145739,7805,9169,2427319.0,1.0,145740
153459,153459,8077,9169,2583445.0,1.0,153460


In [48]:
edge_raw_features.shape, graph_df.shape

((157475, 172), (157474, 6))

In [121]:
def EL_sparsify(graph_df, edge_raw_features):
    """_summary_

    Args:
        graph_df (_type_): Original graph df
        edge_raw_features (_type_): Original edge raw features

    Returns:
        _type_: sparsified graph with edge features
    """
    tmp_graph_df = graph_df.copy(deep=True)
    tmp_graph_df = tmp_graph_df.sort_values(by=['u', 'i', 'ts'])
    
    # Group by 'u' and 'i' and capture the first and last interactions
    first_interactions = tmp_graph_df.groupby(['u', 'i']).first().reset_index()
    last_interactions = tmp_graph_df.groupby(['u', 'i']).last().reset_index()
    
    EL_graph_df = pd.concat([first_interactions, last_interactions]).reset_index(drop=True).drop(['Unnamed: 0'], axis=1)
    
    EL_edge_raw_features = edge_raw_features[EL_graph_df['idx'].values]
    assert EL_graph_df.shape[0] == EL_edge_raw_features.shape[0]
    
    return EL_graph_df, EL_edge_raw_features

In [122]:
new_graph_df, new_edge_raw_features = EL_sparsify(graph_df, edge_raw_features)

In [124]:
new_graph_df.to_csv('./test.csv')

In [120]:
new_graph_df.drop(['Unnamed: 0'], axis=1)

,u,i,ts,label,idx
0,1,8228,0.0,0.0,1
1,2,8229,36.0,0.0,2
2,3,8230,131.0,0.0,4
3,4,8231,169.0,0.0,7
4,5,8232,218.0,0.0,9
...,...,...,...,...,...
36509,8223,8556,2675481.0,1.0,157378
36510,8224,8529,2675530.0,0.0,157382
36511,8225,9206,2677219.0,0.0,157435
36512,8226,8938,2677664.0,0.0,157447


In [125]:
test_df = pd.read_csv('./test.csv')
test_df

,Unnamed: 0,u,i,ts,label,idx
0,0,1,8228,0.0,0.0,1
1,1,2,8229,36.0,0.0,2
2,2,3,8230,131.0,0.0,4
3,3,4,8231,169.0,0.0,7
4,4,5,8232,218.0,0.0,9
...,...,...,...,...,...,...
36509,36509,8223,8556,2675481.0,1.0,157378
36510,36510,8224,8529,2675530.0,0.0,157382
36511,36511,8225,9206,2677219.0,0.0,157435
36512,36512,8226,8938,2677664.0,0.0,157447


In [89]:
tmp_graph_df = graph_df.copy(deep=True)


In [90]:
tmp_graph_df = tmp_graph_df.sort_values(by=['u', 'i', 'ts'])
tmp_graph_df.head()

,Unnamed: 0,u,i,ts,label,idx
0,0,1,8228,0.0,0.0,1
1,1,2,8229,36.0,0.0,2
2,2,2,8229,77.0,0.0,3
4,4,2,8229,150.0,0.0,5
7,7,2,8229,217.0,0.0,8


In [91]:
# Group by 'u' and 'i' and capture the first and last interactions
first_interactions = tmp_graph_df.groupby(['u', 'i']).first().reset_index()
last_interactions = tmp_graph_df.groupby(['u', 'i']).last().reset_index()

In [92]:
first_interactions.shape, last_interactions.shape, tmp_graph_df.shape

((18257, 6), (18257, 6), (157474, 6))

In [93]:
EL_graph_df = pd.concat([first_interactions, last_interactions]).drop_duplicates().reset_index(drop=True)
EL_graph_df


,u,i,Unnamed: 0,ts,label,idx
0,1,8228,0,0.0,0.0,1
1,2,8229,1,36.0,0.0,2
2,3,8230,3,131.0,0.0,4
3,4,8231,6,169.0,0.0,7
4,5,8232,8,218.0,0.0,9
...,...,...,...,...,...,...
27070,8204,8261,156983,2666399.0,0.0,156984
27071,8211,8455,157058,2668424.0,0.0,157059
27072,8215,8251,157251,2672770.0,0.0,157252
27073,8219,9223,157352,2674865.0,0.0,157353


In [63]:
first_interactions['u'].shape, last_interactions['u'].shape, len(first_interactions['u'].unique()), len(last_interactions['u'].unique())

((18257,), (18257,), 8227, 8227)

In [72]:
FI_dict = first_interactions['u'].value_counts().to_dict()
LI_dict = last_interactions['u'].value_counts().to_dict()

In [73]:
FI_dict == LI_dict

True

In [94]:
edge_raw_features[EL_graph_df['idx'].values].shape, EL_graph_df.shape

((27075, 172), (27075, 6))

In [83]:
EL_graph_df['label'].unique()

array([0., 1.])

In [95]:
EL_graph_df[EL_graph_df['label'] == 1.]

,u,i,Unnamed: 0,ts,label,idx
361,33,8260,87,2053.0,1.0,88
4567,689,8592,3773,95206.0,1.0,3774
4774,744,8590,4175,106655.0,1.0,4176
5001,799,8249,9887,231012.0,1.0,9888
6011,1073,8298,15086,324741.0,1.0,15087
...,...,...,...,...,...,...
26929,7822,8345,148622,2484719.0,1.0,148623
26933,7830,8603,146402,2442825.0,1.0,146403
26938,7846,8287,146985,2454297.0,1.0,146986
26974,7938,8703,150004,2510469.0,1.0,150005


In [87]:
first_interactions[first_interactions['label'] == 1.]

,u,i,Unnamed: 0,ts,label,idx
361,33,8260,87,2053.0,1.0,88
4567,689,8592,3773,95206.0,1.0,3774
4774,744,8590,4175,106655.0,1.0,4176
5001,799,8249,9887,231012.0,1.0,9888
6011,1073,8298,15086,324741.0,1.0,15087
...,...,...,...,...,...,...
18078,8053,8594,152810,2572050.0,1.0,152811
18086,8061,8607,153100,2576440.0,1.0,153101
18102,8077,9169,153459,2583445.0,1.0,153460
18135,8109,8670,154390,2606859.0,1.0,154391


In [88]:
last_interactions[last_interactions['label'] == 1.]

,u,i,Unnamed: 0,ts,label,idx
40,15,8241,117582,1970618.0,1.0,117583
361,33,8260,87,2053.0,1.0,88
1266,113,8298,15061,324482.0,1.0,15062
1363,138,8342,701,17601.0,1.0,702
1683,205,8379,132859,2200043.0,1.0,132860
...,...,...,...,...,...,...
18078,8053,8594,152810,2572050.0,1.0,152811
18086,8061,8607,153100,2576440.0,1.0,153101
18102,8077,9169,153459,2583445.0,1.0,153460
18135,8109,8670,154390,2606859.0,1.0,154391


In [96]:
graph_df[graph_df['label'] == 1.]

,Unnamed: 0,u,i,ts,label,idx
87,87,33,8260,2053.0,1.0,88
701,701,138,8342,17601.0,1.0,702
2571,2571,516,8424,68384.0,1.0,2572
3773,3773,689,8592,95206.0,1.0,3774
3801,3801,690,8592,96027.0,1.0,3802
...,...,...,...,...,...,...
153100,153100,8061,8607,2576440.0,1.0,153101
153442,153442,7539,8505,2583015.0,1.0,153443
153459,153459,8077,9169,2583445.0,1.0,153460
154390,154390,8109,8670,2606859.0,1.0,154391


In [38]:
tmp_graph_df_u2 = graph_df[graph_df['u'] == 2]
tmp_graph_df_u2.head()

,Unnamed: 0,u,i,ts,label,idx
1,1,2,8229,36.0,0.0,2
2,2,2,8229,77.0,0.0,3
4,4,2,8229,150.0,0.0,5
7,7,2,8229,217.0,0.0,8
11,11,2,8229,300.0,0.0,12


In [40]:
tmp_graph_df_u2['i'].unique(), tmp_graph_df_u2['label'].unique()

(array([8229]), array([0.]))

In [97]:
graph_df.shape, node_raw_features.shape, edge_raw_features.shape

((157474, 6), (9228, 172), (157475, 172))

In [99]:
EL_graph_df.shape, node_raw_features.shape, edge_raw_features[EL_graph_df['idx'].values].shape

((27075, 6), (9228, 172), (27075, 172))

In [103]:
new_edge_raw_features = edge_raw_features[EL_graph_df['idx'].values]

In [106]:
new_edge_raw_features[0].shape

(172,)

In [108]:
graph_df

,Unnamed: 0,u,i,ts,label,idx
0,0,1,8228,0.0,0.0,1
1,1,2,8229,36.0,0.0,2
2,2,2,8229,77.0,0.0,3
3,3,3,8230,131.0,0.0,4
4,4,2,8229,150.0,0.0,5
...,...,...,...,...,...,...
157469,157469,2004,8860,2678155.0,0.0,157470
157470,157470,3763,9026,2678158.0,0.0,157471
157471,157471,2400,8723,2678293.0,0.0,157472
157472,157472,7480,9148,2678333.0,0.0,157473


In [110]:
EL_graph_df

,u,i,Unnamed: 0,ts,label,idx
0,1,8228,0,0.0,0.0,1
1,2,8229,1,36.0,0.0,2
2,3,8230,3,131.0,0.0,4
3,4,8231,6,169.0,0.0,7
4,5,8232,8,218.0,0.0,9
...,...,...,...,...,...,...
27070,8204,8261,156983,2666399.0,0.0,156984
27071,8211,8455,157058,2668424.0,0.0,157059
27072,8215,8251,157251,2672770.0,0.0,157252
27073,8219,9223,157352,2674865.0,0.0,157353


In [100]:
len(EL_graph_df['u'].unique())

8227

In [101]:
len(EL_graph_df['i'].unique())

1000

In [102]:
node_raw_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [1]:
tmp_graph_df

NameError: name 'tmp_graph_df' is not defined